In [1]:
import sys

import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.core.display import HTML

import random_proteome as randp
AMINO_ACID_LST = randp.AMINO_ACID_LST

# Random proteome creator

In [2]:
upload_button = widgets.FileUpload(
    button_style="info",
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


output_log = widgets.Output(layout={'border': '1px solid #e5e8e8', 
                                    'padding': '3px',
                                    'color': '#515a5a'})

with output_log:
    print("Click on the button above to upload fasta file.")
    print("Then, click on 'Create random proteome'.")

In [3]:
template="template-proteome.fasta"
@output_log.capture(clear_output=True)
def on_file_upload(change):
    with output_log:
        if upload_button.value:
            [uploaded_file] = upload_button.value
            with open(template, "wb") as fp:
                fp.write(upload_button.value[uploaded_file]["content"])
            print(f"File uploaded succesfuly!")
        else:
            print("Upload file first!")

upload_button.observe(on_file_upload, "value")

In [4]:
process_button = widgets.Button(description="Create random proteome",
                                layout={"width": "250px"},
                                button_style="info",)

@output_log.capture(clear_output=True)
def on_click_process_button(b):
    PROTEIN_NAME_LST, PROTEIN_LEN_LST, PROTEOME_SEQ = randp.read_template_proteome(template)
    print(f"Read {len(PROTEIN_NAME_LST)} proteins.")
    print(f"Found {len(PROTEOME_SEQ)} amino acids.")
    print("")
    PROTEOME_AMINO_ACID_DISTRIBUTION = randp.get_amino_acid_proportion(PROTEOME_SEQ, 
                                                                       randp.AMINO_ACID_LST)
    shuffled_proteome_sequence = randp.shuffle_sequence(PROTEOME_SEQ)
    print("\nBuilding random proteins...")
    random_protein_seq_lst = randp.create_random_proteins_from_proteome(
                                                                shuffled_proteome_sequence,
                                                                PROTEIN_LEN_LST)
    fasta_filename = "random_proteome.fasta"
    randp.write_fasta(random_protein_seq_lst, fasta_filename)
    print(f"{len(random_protein_seq_lst)} proteins created!")
    print("\nComputing amino acid distribution probability...")
    tsv_filename = "random_proteome.tsv"
    randp.write_distribution(random_protein_seq_lst, 
                             randp.AMINO_ACID_LST, 
                             tsv_filename, 
                             ref_distribution=PROTEOME_AMINO_ACID_DISTRIBUTION)
    print(f"Saved!")
    display(HTML(f"<a download='{fasta_filename}' href='{fasta_filename}'>Click here to download random proteome</a>"))
    display(HTML(f"<a download='{tsv_filename}' href='{tsv_filename}'>Click here to download random proteome amino acid distribution</a>"))

process_button.on_click(on_click_process_button)

In [5]:
display(widgets.HBox([upload_button, process_button]))
display(output_log)

Output(layout=Layout(border='1px solid #e5e8e8', padding='3px'))